In [11]:
import pandas as pd
import numpy as np

import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))
from utils import load_data

In [12]:
df = load_data('../data/fuel_consumption.csv')

features = ['make', 'release_year', 'vehicle_class', 'fuel_type', 'transmission_type', 'gears', 'engine_size', 'cylinders']
numerical_features = ['release_year', 'gears', 'engine_size', 'cylinders']
categorical_features = ['make', 'vehicle_class', 'fuel_type', 'transmission_type']

targets = ['emissions', 'fc_mixed', 'fc_city', 'fc_highway']

X = df[features]
Y = df[targets]

### Model definition

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso

preprocessor = ColumnTransformer(
    transformers=[
        ('categorical', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('numerical', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), numerical_features)
    ])

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Lasso(max_iter=10000))
])

param = {
    'regressor__alpha': np.logspace(-4, 4, 5)
}

In [ ]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(pipeline, param, cv=5, n_jobs=-1, refit = True)
grid.fit(X, Y)

print(f"Best parameters : {grid.best_params_}")
print(f"Best score : {grid.best_score_:0.2f}")

/Users/mathisderenne/miniconda3/envs/FuelConsumption/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:658: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 214518.64188108686, tolerance: 6285.037067451783
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/mathisderenne/miniconda3/envs/FuelConsumption/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:658: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 260871.4579803981, tolerance: 6595.572975871425
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/mathisderenne/miniconda3/envs/FuelConsumption/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:658: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1109974.1843254603, tolerance: 6387.9281911665375
  model = cd_fast.

Best parameters : {'regressor__alpha': 0.0001}
Best score : 0.7686658407887057


### Save model

In [16]:
import pickle

with open('../data/lasso_regression.pkl', 'wb') as model_file:
    pickle.dump(grid.best_estimator_, model_file)